#Protein Language Visualization (PLVis) Colab Notebook
Use this Colab Notebook to generate your own PLVis for proteome comparisons.

In [ ]:
#@title Step 1. Import Dependencies
#@markdown Run this cell to import all the necessary dependencies.

import pandas as pd
import numpy as np
np.random.seed(77)
import more_itertools
import os
import io
import re
import h5py
import gzip
import requests
import PIL
import itertools
import random
import seaborn as sns
import urllib.request
import urllib.error

import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib.colors as mcolors

from collections import Counter
from fontTools import ttLib

from datetime import datetime
current_date = datetime.now().date()
date = current_date.strftime('%Y-%m-%d')

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances

from nltk import ngrams, FreqDist

from scipy.spatial.distance import cdist

from tqdm import tqdm

!pip install umap-learn --quiet
import umap

!pip install datamapplot --quiet
import datamapplot

!pip install ipywidgets --quiet
import ipywidgets as widgets
from IPython.display import display

!pip install graphein --quiet
!pip install graphein[extras] --quiet
from graphein.protein.utils import download_alphafold_structure

!pip install py3Dmol --quiet
import py3Dmol

from google.colab import files
from google.colab import drive

print('Imported dependencies')

#DMP FUNCTIONS
def get_data(df):
    data_map = df[['UMAP 1', 'UMAP 2']].to_numpy().astype('float32')

    data_list = []
    for index, row in df.iterrows():
        entry = row['Entry'] if 'Entry' in row else 'N/A'
        protein = row['Protein names'] if 'Protein names' in row else 'N/A'
        organism = row['Organism'] if 'Organism' in row else 'N/A'
        gene = row['Gene Names'] if 'Gene Names' in row else 'N/A'
        pathway = row['Pathway'] if 'Pathway' in row else 'N/A'
        anot = row['Annotation'] if 'Annotation' in row else 'N/A'
        label = row['Cluster Label']
        text = f"Entry: {entry}\nProtein Names: {protein}\nOrganism: {organism}\nGene Names: {gene}\nPathway: {pathway}\nAnnotation: {anot}\nCluster: {label}"
        data_list.append(text)
    hover_data = np.array(data_list).astype(str)

    clusters = sorted(df['Cluster Label'].unique())
    name_map = {}
    for cluster in clusters:
        current_cluster = df[df['Cluster Label'] == cluster]
        prot_names = current_cluster['Protein names'].tolist()
        prot_names = [re.sub(r'\([^)]*\)|\[[^\]]*\]', '', str_test) for str_test in prot_names]
        words = [name.split() for name in prot_names]
        word_list = [item for sublist in words for item in sublist]
        dict_counts = FreqDist(ngrams(word_list, 2))

        if len(dict_counts)==0:
            max_counts = ('No', '', 'label',)
        else:
            max_counts = max(dict_counts, key=dict_counts.get)

        cluster_name = ' '.join(max_counts)
        name_map[cluster] = f'{cluster}. {cluster_name}'

    df['Cluster Name'] = df['Cluster Label'].map(name_map)
    labels = (df['Cluster Name']).to_numpy().astype(str)

    return(data_map, hover_data, labels)

def get_customs(color_mapping):
    custom_css="""
    .row {
        display : flex;
        align-items : center;
    }
    .box {
        height:10px;
        width:10px;
        border-radius:2px;
        margin-right:5px;
    }
    #legend {
        position: absolute;
        bottom: 0;
        right: 0;
        margin: 16px;
        padding: 12px;
        border-radius: 16px;
        z-index: 2;
        background: #ffffffcc;
        font-family: Cinzel;
        font-size: 8pt;
        box-shadow: 2px 3px 10px #aaaaaa44;
    }
    #title-container {
        max-width: 75%;
    }
    """
    custom_html = """
    <div id="legend">
    """
    for field, color in color_mapping.items():
        custom_html += f'    <div class="row"><div id="{field}" class="box" style="background-color:{color};padding:0px 0 1px 0;text-align:center"></div>{field}</div>\n'
    custom_html += """
    </div>
    """
    custom_js = """
        const legend = document.getElementById('legend');
        legend.addEventListener('click', function (event) {
                const primary_field = event.srcElement.id;
                selectPoints(primary_field, (i) => (hoverData.data.primary_field[i] == primary_field));
                for (const row of legend.children) {
                    for (const item of row.children) {
                        if (item.id == primary_field) {
                            item.innerHTML = "✓";
                        } else {
                            item.innerHTML = "";
                        }
                    }
                }
                search.value = "";
            });

            search.addEventListener("input", (event) => {
                for (const row of legend.children) {
                    for (const item of row.children) {
                            item.innerHTML = "";
                        }
                    }
            });
    """
    return(custom_css, custom_html, custom_js)

**To use the PLVis Colab Notebook, you will need the following files for each protein set:**

1) A CSV, TSV or XLSX dataframe from UniProt with the following columns:
* "Entry", "Sequence" and "Annotation"

2) A GZ or H5 file containing the corresponding embeddings for the dataframe.

---
**IMPORTANT**

Be sure to give the same name to your dataframe and embedding files before uploading.

---

**NOTE**

If you have already generated a dataframe using this Colab, skip steps 2 through 6.

In [ ]:
#@title Step 2. Upload Dataframe and Embedding Files

#@markdown Run this cell and upload ALL your protein dataframe files
#@markdown (TSV, CSV or XLSX) and embedding files (GZ or H5).

# Prompt user to upload multiple files
print("Please upload your protein dataframe files (TSV, CSV or XLSX) and embeddings (GZ or H5):")
uploaded_files = files.upload()  # Allow multiple file uploads

# Initialize the file dictionary
file_dict = {}

# Process uploaded files
for file_name in uploaded_files.keys():
    # Save uploaded files locally
    with open(file_name, 'wb') as f:
        f.write(uploaded_files[file_name])

# Group files by their base names
file_groups = {}
for file_name in uploaded_files.keys():
    base_name = os.path.splitext(file_name)[0]  # Extract base name without extension
    if base_name not in file_groups:
        file_groups[base_name] = []
    file_groups[base_name].append(file_name)

# Match data files with embeddings and process them
for base_name, file_names in file_groups.items():
    df_file = None
    embedding_file = None
    compressed = None

    # Check for dataframe files
    for file_name in file_names:
        if file_name.endswith('.tsv'):
            try:
                df_file = pd.read_csv(file_name, sep='\t')
                print(f"Loaded dataframe from {file_name}.")
            except Exception as e:
                print(f"Error reading {file_name} as TSV: {e}")
        elif file_name.endswith('.csv'):
            try:
                df_file = pd.read_csv(file_name)
                print(f"Loaded dataframe from {file_name}.")
            except Exception as e:
                print(f"Error reading {file_name} as CSV: {e}")
        elif file_name.endswith('.xlsx'):
            try:
                df_file = pd.read_excel(file_name)
                print(f"Loaded dataframe from {file_name}.")
            except Exception as e:
                print(f"Error reading {file_name} as XLSX: {e}")

        # Check for embedding files
        elif file_name.endswith('.gz'):
            embedding_file = file_name
            compressed = True
            print(f"Found compressed embedding file: {file_name}.")
        elif file_name.endswith('.h5'):
            embedding_file = file_name
            compressed = False
            print(f"Found H5 embedding file: {file_name}.")

    # Ensure both dataframe and embedding file exist
    if df_file is not None and embedding_file is not None:
        file_dict[base_name] = {
            'df_file': df_file,
            'embedding_file': embedding_file,
            'compressed': compressed
        }
    else:
        print(f"Missing pair for {base_name}. Skipping...")

# Output the results
if file_dict:
    print("\nSuccessfully processed the following file pairs:")
    for key, details in file_dict.items():
        print(f"Base Name: {key}, DataFrame Loaded, Embedding File: {details['embedding_file']}, Compressed: {details['compressed']}")
else:
    print("No valid file pairs were found.")

In [ ]:
#@title Step 3. Retrieve the PLM Embeddings

#@markdown Run this cell to retrieve the protein embeddings from an H5 or GZ file.

def get_embeddings (embedding_file, df_file):
    entries_list = []
    embeddings_list = []
    data_map = {}

    if embedding_file.endswith('.gz'):
        output_file = embedding_file.replace('.gz', '.h5')
        with gzip.open(embedding_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                f_out.write(f_in.read())
    elif embedding_file.endswith('.h5'):
        output_file = embedding_file

    with h5py.File(output_file, "r") as file:
        for sequence_id, embedding in file.items():
            entries_list.append(sequence_id)
            embeddings = np.array(embedding)
            embeddings_list.append(embeddings)
            data_map[sequence_id] = embeddings

    embedding_df = df_file
    embedding_df['Embeddings'] = embedding_df['Entry'].map(data_map)

    #Remove proteins without embeddings from the dataframe
    embedding_df = embedding_df.dropna(subset=['Embeddings'])

    return(embedding_df)

#Updated dfs with the embeddings for each protein
df_list = []
for file_name in file_dict:
    embedding_df = get_embeddings(file_dict[file_name]['embedding_file'],
                                  file_dict[file_name]['df_file'])
    df_list.append(embedding_df)

embedding_df = pd.concat(df_list, ignore_index=True)
print(f"Retrieved {len(embedding_df)} embeddings")

In [ ]:
#@title Step 4. Reduce Dimensionality

#@markdown This cell uses the **UMAP** reduction algorithm to reduce the PLM
#@markdown embeddings to 2 dimensions.

#@markdown The time it takes for this cell to run depends on the
#@markdown size of your dataframe.

try:
    embeddings_array = np.array(embedding_df['Embeddings'].tolist())

    umap_reducer = umap.UMAP(n_components = 2, random_state = 77)

    embeddings_reduced = umap_reducer.fit_transform(embeddings_array)

    reduced_df = embedding_df.copy()
    reduced_df['UMAP 1'] = embeddings_reduced[:, 0]
    reduced_df['UMAP 2'] = embeddings_reduced[:, 1]

    print("2D reduction succesful")
except Exception as e:
    print(f"Error reducing the embeddings: {e}")

In [ ]:
#@title Step 5. Calculate the Number of Clusters

#@markdown This cell calculates the number of k-means clusters for the
#@markdown dataframe. The plot generated indicates the average silhouette
#@markdown score for the number of clusters in the visualization, a higher score
#@markdown means that the clusters are better defined.

#@markdown The time it takes for this cell to run depends on the
#@markdown size of your dataframe.

#@markdown ---

#@markdown The default range for this analysis is 2-100 k-means clusters.
#@markdown If you wish to use a different upper limit, write the number in the space below.

custom_range = "" #@param{type:"string"}
if custom_range == "":
    max_range = 101
else:
    max_range = int(custom_range) + 1
cluster_range = range(2, max_range)

def optimal_clusters(embeddings_2d, cluster_range):
    sil_scores_kmeans = []

    for n_clusters in tqdm(cluster_range):
        kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
        kmeans_labels = kmeans.fit_predict(embeddings_2d)
        sil_score_kmeans = silhouette_score(embeddings_2d, kmeans_labels)
        sil_scores_kmeans.append(sil_score_kmeans)

    optimal_clusters =  cluster_range[np.argmax(sil_scores_kmeans)]

    plt.figure(figsize=(16, 10))
    plt.plot(cluster_range, sil_scores_kmeans, markersize=5)

    plt.xlabel('Number of K-means Clusters', fontsize=12, fontweight='bold')
    plt.ylabel('Silhouette Score', fontsize=12, fontweight='bold')
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.show()
    return(optimal_clusters)

#@markdown ---
#@markdown Check this box if you wish to perform clustering in high dimensions.
high_dimension_clustering = False #@param {type:"boolean"}
if high_dimension_clustering:
  embeddings_nd= np.stack(reduced_df['Embeddings'].to_numpy())
  kclusters = optimal_clusters(embeddings_nd, cluster_range)
else:
  embeddings_2d = reduced_df[['UMAP 1', 'UMAP 2']].to_numpy()
  kclusters = optimal_clusters(embeddings_2d, cluster_range)

print(f'Optimal clusters: {kclusters}')

In [ ]:
#@title Step 6. Save the DataFrame

#@markdown This cell saves your dataframe and automatically downloads it to your computer.
#@markdown If you do not wish to use the optimal number of clusters calculated in the previous step,
#@markdown you can change the number of k-clusters using the space below.

#@markdown Please indicate the name for the generated PLVis dataframe.

file_name = "" #@param {type:"string"}
if file_name == "":
    file_name = f"plvis"

#@markdown ---
#@markdown Leave this space empty if you wish to use the optimal number of k-clusters.

personalized_clusters = "" #@param {type:"string"}
if personalized_clusters == "":
    kclusters = kclusters
else:
    kclusters = int(personalized_clusters)

clusters = KMeans(kclusters, n_init=10, random_state=42)

complete_df = reduced_df.copy()

#@markdown ---
#@markdown Check this box if you wish to perform clustering in high dimensions.
high_dimension_clustering = False #@param {type:"boolean"}
if high_dimension_clustering:
  complete_df['Cluster Label'] = clusters.fit_predict(embeddings_nd)
else:
  complete_df['Cluster Label'] = clusters.fit_predict(complete_df[['UMAP 1', 'UMAP 2']])

complete_df = complete_df.drop(columns=['Embeddings'])
complete_df.to_csv(f"{file_name}_{date}.csv", index=False)
files.download(f"{file_name}_{date}.csv")

---

In [ ]:
#@title Load a Previously Generated Dataframe

#@markdown **NOTE: Skip this cell if you created the PLVis dataframe for the first time.**

#@markdown Run this cell to upload a previously generated PLVis dataframe.

# Prompt the user to upload a single file
print("Please upload a single file (CSV format).")
uploaded_file = files.upload()

# Check if exactly one file is uploaded
if len(uploaded_file) != 1:
    print("Error: Please upload only one file.")
else:
    # Get the file name
    file_name = list(uploaded_file.keys())[0]
    try:
        # Try reading the file as a CSV
        complete_df = pd.read_csv(file_name)

        # Check if the required columns are present
        required_columns = ['UMAP 1', 'UMAP 2', 'Cluster Label']
        if all(col in complete_df.columns for col in required_columns):
            print(f"The file '{file_name}' is a valid PLVis dataframe.")
        else:
            missing_columns = [col for col in required_columns if col not in complete_df.columns]
            print(f"The file '{file_name}' is missing the following required columns: {missing_columns}")

    except Exception as e:
        # Handle errors (e.g., file is not a valid CSV)
        print(f"Error processing the file '{file_name}': {e}")

---

In [ ]:
#@title PLVis

#@markdown The following projection is a quick view of the complete dataframe.
#@markdown Each color represents a different cluster in the data. The
#@markdown names for each cluster are generated using n-gram analysis of the
#@markdown protein names within the data.

#@markdown Please indicate the title to be viewed with the projection.

projection_df = complete_df.copy()
data_map, hover_data, labels = get_data(projection_df)

plvis_title = "" #@param {type:"string"}
if plvis_title == "":
    plvis_title = "Cluster View"

plot = datamapplot.create_interactive_plot(
    data_map,
    labels,
    font_family="Cinzel",
    title=plvis_title,
    sub_title="PLVis Project",
    hover_text=hover_data,
    enable_search=True
)
plot.save(f"{plvis_title}.html")
plot

---

In [ ]:
#@title Column Analysis
#@markdown This next section allows the user to compare between two different
#@markdown sets of data within the dataframe. To start, please run the cell and
#@markdown select the column that you wish to analyze.

cols_to_remove = ['Entry','Entry Name', 'Protein names', 'Sequence',
                  'Length', 'UMAP 1', 'UMAP 2']

cols = [col for col in complete_df.columns if col not in cols_to_remove]
cols.insert(0, ' ')

def selected_col(change):
    global sel_col
    sel_col = col_sel.value
    if sel_col == ' ':
        print("WARNING: please select a valid column")
    else:
        print('Selected ', sel_col)

col_sel = widgets.Dropdown(options=cols,
                        description='Select Column:')
col_sel.observe(selected_col, names='value')

widgets.VBox([col_sel])

In [ ]:
#@title Comparison Selection
#@markdown Please run this cell and select the
#@markdown comparison that you wish to study from the dropdown menu.

options = complete_df[sel_col].unique()

combinations = list(itertools.combinations(options, 2))
combination_options = {f"{a} vs {b}": (a, b) for a, b in combinations}
combination_options = {' ': None, **combination_options}

def selected_comp(change):
    global sel_comp
    sel_comp = comp_sel.value
    if sel_comp is None:
        print("WARNING: please select a valid comparison")
    else:
        print('Selected ', sel_comp)

comp_sel = widgets.Dropdown(options=combination_options,
                            description='Select Comparison:')
comp_sel.observe(selected_comp, names='value')

widgets.VBox([comp_sel])

In [ ]:
#@title Comparison Visualization

#@markdown Run this cell to visualize your comparison. Each entry studied is
#@markdown colored in a different color to distinguish them.

comp_df = complete_df
comp_df = comp_df[comp_df[sel_col].isin(sel_comp)]
data_map, hover_data, labels = get_data(comp_df)

color_mapping = {sel_comp[0]: 'darkred',
                 sel_comp[1]: 'darkblue'}

marker_color_array = pd.Series(comp_df[sel_col]).map(color_mapping).values
custom_css, custom_html, custom_js = get_customs(color_mapping)

plvis_title = "" #@param {type:"string"}
if plvis_title == "":
    plvis_title = "Comparison View"

plvis_comp = datamapplot.create_interactive_plot(
    data_map,
    font_family="Cinzel",
    title=plvis_title,
    sub_title="PLVis project",
    hover_text=hover_data,
    enable_search=True,
    color_label_text=False,
    marker_color_array=marker_color_array,
    custom_css=custom_css,
    custom_html=custom_html)

plvis_comp.save(f"{plvis_title}.html")
plvis_comp

---

In [ ]:
#@title AlphaFold Structure Comparison
#@markdown In this section you can visualize the available AlphaFold generated structures for
#@markdown proteins within a cluster. Please run this cell and select a cluster
#@markdown label from the dropdown menu to start.

clus_labels = np.sort(complete_df['Cluster Label'].unique()).tolist()
clus_labels.insert(0, ' ')

def selected_clus(change):
    global sel_clus
    sel_clus = clus_sel.value
    if sel_clus == ' ':
        print("WARNING: please select a valid column")
    else:
        print('Selected ', sel_clus)

clus_sel = widgets.Dropdown(options=clus_labels,
                            description='Cluster:')
clus_sel.observe(selected_clus, names='value')

widgets.VBox([clus_sel])

In [ ]:
#@title Retrieve the AlphaFold Structures
#@markdown This cell downloads all the corresponding PDB files for the AlphaFold
#@markdown structures of the proteins in the selected cluster.

cluster_df = complete_df[complete_df['Cluster Label'] == 	sel_clus]

pdb_dir = "alphafold_structures"
if not os.path.exists(pdb_dir):
    os.makedirs(pdb_dir)

missing = []
downloaded = []
print(f'Retrieving {len(cluster_df)} files')
for index, row in cluster_df.iterrows():
    key = row['Entry']
    pdb_file = os.path.join(pdb_dir, f'{key}.pdb')
    if not os.path.exists(pdb_file):
        download_alphafold_structure(key, out_dir = pdb_dir, aligned_score=False)
        if not os.path.exists(pdb_file):
            missing.append(key)
        else:
            downloaded.append(key)

print(f'Successfully downloaded {len(downloaded)} files')
print(f'Missing {len(missing)} files')

In [ ]:
#@title Structure Comparison

#@markdown In this cell you can visualize a random selection of 9 proteins from
#@markdown the selected cluster. Click on the viewer to rotate them.

if not downloaded:
    print("No available structures.")
else:

    if len(downloaded) >= 9:
        entries_list = random.sample(downloaded, 9)
    else:
        entries_list = downloaded

    protein_list = []
    for entry in entries_list:
        try:
            with open(f'alphafold_structures/{entry}.pdb', 'r') as file:
                content = file.read()
                protein_list.append(content)
        except:
            print(f'{entry} not available')

    view = py3Dmol.view(viewergrid=(3,3), width=800, height=500, linked=True)
    view.setViewStyle({'style':'outline','color':'k','width':0.1})
    view.setBackgroundColor('black')

    for i in range(len(protein_list)):
        row = i//3
        col = i%3
        view.addModel(protein_list[i],'pdb',viewer=(row,col))
        view.setStyle({"cartoon": {'color': 'spectrum'}},viewer=(row,col))
        view.zoomTo(viewer=(row,col))

        title = entries_list[i] if i < len(entries_list) else f"Protein {i+1}"
        view.addLabel(title, {'fontColor':'white', 'fontSize':12, 'position': {'x':0, 'y':5, 'z':0}}, viewer=(row, col))

    view.show()